In [155]:
using HDF5
using ITensors

In [156]:
function load_complex_dataset_as_array(file_path, dataset_name)
    file = h5open(file_path, "r")
    dataset = read(file[dataset_name]) 
    close(file)
    
    # Initialise an array to hold the complex numbers, without predefining a shape
    complex_array = [complex(d.real, d.imag) for d in dataset]
    
    return complex_array
end

load_complex_dataset_as_array (generic function with 1 method)

In [157]:
s1 = load_complex_dataset_as_array("complex_data.h5", "site1");
s2 = load_complex_dataset_as_array("complex_data.h5", "site2");

In [158]:
s2[1]

-0.8357146382331848 - 1.3114923238754272im

In [159]:
s1_final = Array{ComplexF64}(undef, 1, 2, 4)
s1_final[1, 1, 1] = s1[1]
s1_final[1, 1, 2] = s1[2]
s1_final[1, 1, 3] = s1[3]
s1_final[1, 1, 4] = s1[4]

s1_final[1, 2, 1] = s1[5]
s1_final[1, 2, 2] = s1[6]
s1_final[1, 2, 3] = s1[7]
s1_final[1, 2, 4] = s1[8]

0.8082578182220459 - 0.1832321584224701im

In [160]:
s2_final = Array{ComplexF64}(undef, 4, 2, 1);
s2_final[1, 1, 1] = s2[1];
s2_final[1, 2, 1] = s2[2];

s2_final[2, 1, 1] = s2[3];
s2_final[2, 2, 1] = s2[4];

s2_final[3, 1, 1] = s2[5];
s2_final[3, 2, 1] = s2[6];

s2_final[4, 1, 1] = s2[7];
s2_final[4, 2, 1] = s2[8];


In [161]:
s2_final

4×2×1 Array{ComplexF64, 3}:
[:, :, 1] =
 -0.835715-1.31149im    0.420888+0.579194im
  0.510843-1.20403im   -0.225118+1.28012im
   0.40536-0.28335im   -0.137874+0.106533im
   1.07665+0.999973im   0.687609-0.615196im

Make into ITensors

In [162]:
s = siteinds("S=1/2", 2)

2-element Vector{Index{Int64}}:
 (dim=2|id=249|"S=1/2,Site,n=1")
 (dim=2|id=27|"S=1/2,Site,n=2")

In [163]:
reshape(s2_final, (4, 2))

4×2 Matrix{ComplexF64}:
 -0.835715-1.31149im    0.420888+0.579194im
  0.510843-1.20403im   -0.225118+1.28012im
   0.40536-0.28335im   -0.137874+0.106533im
   1.07665+0.999973im   0.687609-0.615196im

In [164]:
site_1_idx = Index(2, "S=1/2,Site,n=1");
link_1_idx = Index(4, "Link,l=1");
dummy_index = Index(1, "dummy");

s1_tensor = ITensor(s1_final, dummy_index, site_1_idx, link_1_idx)

ITensor ord=3 (dim=1|id=10|"dummy") (dim=2|id=937|"S=1/2,Site,n=1") (dim=4|id=264|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [165]:
site_2_idx = Index(2, "S=1/2,Site,n=2")
s2_tensor = ITensor(s2_final, dummy_index, site_2_idx, link_1_idx)

ITensor ord=3 (dim=1|id=10|"dummy") (dim=2|id=908|"S=1/2,Site,n=2") (dim=4|id=264|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

Combine into an MPS

In [166]:
mps = MPS([s1_tensor, s2_tensor])

MPS
[1] ((dim=1|id=10|"dummy"), (dim=2|id=937|"S=1/2,Site,n=1"), (dim=4|id=264|"Link,l=1"))
[2] ((dim=1|id=10|"dummy"), (dim=2|id=908|"S=1/2,Site,n=2"), (dim=4|id=264|"Link,l=1"))


In [167]:
@show mps[1]

mps[1] = ITensor ord=3
Dim 1: (dim=1|id=10|"dummy")
Dim 2: (dim=2|id=937|"S=1/2,Site,n=1")
Dim 3: (dim=4|id=264|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 1×2×4
[:, :, 1] =
 -0.6767696738243103 - 0.928959310054779im  0.21445874869823456 - 0.9110893607139587im

[:, :, 2] =
 -0.28865286707878113 - 0.4180883765220642im  0.22639237344264984 + 0.1902822107076645im

[:, :, 3] =
 0.4713106155395508 + 0.3606642186641693im  0.06180282309651375 + 0.8166481852531433im

[:, :, 4] =
 -0.33472883701324463 - 1.3707609176635742im  0.8082578182220459 - 0.1832321584224701im


ITensor ord=3 (dim=1|id=10|"dummy") (dim=2|id=937|"S=1/2,Site,n=1") (dim=4|id=264|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

Let's make some example data using the complex feature map:

In [168]:
function ComplexAngleEncoder(x) 
    """Function to convert normalised time series to an angle encoding."""
    if x <= 1.0 && x >= 0.0
        s1 = exp(1im * (3pi/2)*x) * cospi(0.5 * x)
        s2 = exp(-1im * (3pi/2)*x) * sinpi(0.5 * x)
        return [s1, s2]
    else
        println("Data points must be rescaled between 1 and 0 before encoding using the angle encoder.")
    end
end

ComplexAngleEncoder (generic function with 1 method)

In [169]:
function NormalisedDataToProductState(sample::Vector, site_indices::Vector{Index{Int64}})
    """Function to convert a single normalised sample to a product state
    with local dimension 2, as specified by the feature map."""

    n_sites = length(site_indices) # number of mps sites
    product_state = MPS(site_indices; linkdims=1)
    
    # check that the number of sites matches the length of the time series
    if n_sites !== length(sample)
        error("Number of MPS sites: $n_sites does not match the time series length: $(length(sample))")
    end

    for j=1:n_sites
        T = ITensor(site_indices[j])
        # map 0 to |0> and 1 to |1> 
        zero_state, one_state = ComplexAngleEncoder(sample[j])
        T[1] = zero_state
        T[2] = one_state
        product_state[j] = T
    end

    return product_state

end

NormalisedDataToProductState (generic function with 1 method)

In [175]:
sample = [0, 0]

2-element Vector{Int64}:
 0
 0

In [176]:
ps = NormalisedDataToProductState(sample, siteinds(mps))

MPS
[1] ((dim=2|id=937|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=908|"S=1/2,Site,n=2"),)


In [179]:
@show ps[1]

ps[1] = ITensor ord=1
Dim 1: (dim=2|id=937|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 1.0 + 0.0im
 0.0 - 0.0im


ITensor ord=1 (dim=2|id=937|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [172]:
inner(mps, ps)

1.5502376636186155 - 1.02034104682372im

In [173]:
abs(inner(mps, ps))

1.8558913399051757